# Fractals

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tools.fractools as ft
from tools.fractools import complex
from matplotlib.animation import FuncAnimation
from tqdm import tqdm

# Increase font size of plots
plt.rcParams.update({'font.size': 13.5})

# Automatically reload modules with saved changes
%load_ext autoreload
%autoreload 2

---
## Koch snowflake
The procedure for generating a Koch snowflake starts with a line bound by two points. The line is split into 3 equal segments defined by 4 points, and the central segment is then bent into an angle (conventionally 60 degrees), thereby resulting in a shape consisting of 4 lines. This sequence is applied iteratively to each line, resulting in a Koch fractal, which can be stitched with two rotated copies of itself to form a Koch snowflake.

In [ ]:
# >>> Plot a basic Koch snowflake

# Initial points and test snowflake
x = np.array([-1, 1])
y = np.array([0, 0])
s = ft.koch_snowflake(x, y, 8, np.pi/3)

# Create a square figure and plot the snowflake
fig, ax = plt.subplots(figsize = (5, 5))
ax.plot(s[0], s[1], color = 'b', lw = 1)

# Remove ticks and show plot
ax.set_xticks([]), ax.set_yticks([])
plt.savefig('outputs/snowflake.png', dpi = 300, bbox_inches = 'tight')
plt.show()

In [ ]:
# >>> Make an animation of varying Koch angles

# Specify number of frames
nframes = 300

# Initial line definition
x = np.array([-1, 1])
y = np.array([0, 0])

# Set angle values and calculate a snowflake for each one
thetas = np.linspace(0.01, 1.99*np.pi, nframes)
snowflakes = [ft.koch_snowflake(x, y, 5, theta) for theta in tqdm(thetas)]

# Set temporary formatting
with mpl.rc_context({'lines.linewidth': 2, 'axes.facecolor': 'black'},):

    # Create figure first and format
    fig, ax = plt.subplots(figsize = (5, 5))
    fig.patch.set_alpha(0)

    # Define a frame update function
    def update_koch(frame):
        ax.clear()
        plt.plot(snowflakes[frame][0], snowflakes[frame][1], color = 'b')
        ax.set_xticks([])
        ax.set_yticks([])
        return

    # Save the animation as a gif
    anim = FuncAnimation(fig, update_koch, frames = tqdm(range(nframes)), blit = False)
    anim.save('outputs/koch_snowflake.gif', writer = 'pillow', fps = 20, dpi = 300)

---
## Sierpinski triangle
A Sierpinski triangle is formed by connecting the midpoints of all three edges of an equilateral triangle, thereby dividing it into four equilateral triangles, and repeating indefinitely.

In [ ]:
# >>> Plot a basic Sierpinski triangle

# Create a square figure first
fig, ax = plt.subplots(figsize = (5, 5))

# Initial points and test triangle
triangle = np.array([(-1, -1), (1, -1), (0, 1)])
ft.sierpinski_triangle(ax, triangle, 6, color = 'b', lw = '0.1')

# Remove ticks, set axis limits, and show plot
ax.set_xticks([]), ax.set_yticks([])
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
plt.show()

In [ ]:
# >>> Make an animation of varying Koch angles

# Specify number of frames
nframes = 60

# Set size of steps for zooming in
step_size = 1/nframes

# Create figure first and format
fig, ax = plt.subplots(figsize = (5, 5))
fig.patch.set_alpha(0)
ax.set_xticks([]), ax.set_yticks([])

# Plot all the triangles first
triangle = np.array([[-1, -1], [1, -1], [0, 1]])
ft.sierpinski_triangle(ax, triangle, 8, color = 'b', lw = '0.1')

# Define a frame update function that gradually zooms in
def update_sierpinski(frame):
    plt.xlim(-(1 - frame*step_size), 1)
    plt.ylim(-1, (1 - frame*step_size))
    return

# Save the animation as a gif
anim = FuncAnimation(fig, update_sierpinski, frames = tqdm(range(nframes + 1)), blit = False)
anim.save('outputs/sierpinski.gif', writer = 'pillow', fps = 30, dpi = 300)

---
## Julia sets and the Mandelbrot set
This is essentially an example of how fractals can manifest from simple iterations of complex functions. The iteration in question is taken as $z \rightarrow z^2 + c$, where both $z$ and $c$ are complex numbers. Different Julia sets arise for a constant $c$ and varying starting $z$ points, whereas the Mandelbrot set describes a special case with $z$ starting at the origin and $c$ changing. Whether the sequence diverges or converges determines how the initial point is plotted, which results in very intricate structures.

In [ ]:
# >>> Animate an example Julia set

# Create figure first and format
fig, ax = plt.subplots(figsize = (5, 5))
fig.patch.set_alpha(0)
ax.set_xticks([]), ax.set_yticks([])

# Fractal settings
n_vals = 500 # binning
val_range = np.array([-1.5, 1.5])
c = complex(0, 0.66)

# Animation settings
nframes = 30
fpoint = np.array([0.3, 0.5]) # destination of zoom
steps = (fpoint - val_range) / nframes

# Define a frame update function that gradually zooms in
def julia_zoom(frame):
    vals = np.linspace(val_range[0] + steps[0]*frame, val_range[1] + steps[1]*frame, n_vals)
    d = np.array([[complex.iterate(complex(a, b), c, max_iter = 100, threshold = 10) \
                   for b in vals] for a in vals])
    plt.imshow(d.T, cmap = 'bone')
    return

# Save the animation as a gif
anim = FuncAnimation(fig, julia_zoom, frames = tqdm(range(nframes + 1), position = 0), blit = False)
anim.save('outputs/julia.gif', writer = 'pillow', fps = 15, dpi = 300)

In [ ]:
# >>> Animate an example Mandelbrot set

# Create figure first and format
fig, ax = plt.subplots(figsize = (5, 5))
fig.patch.set_alpha(0)
ax.set_xticks([]), ax.set_yticks([])

# Fractal settings
n_vals = 500 # binning
a_range = np.array([-2.1, 1])
b_range = np.array([-1.5, 1.5])
z = complex(0, 0)

# Animation settings
nframes = 30
fpoint_a = np.array([-0.4, 0.1]) # destination of zoom for a
fpoint_b = np.array([-1.2, -0.7]) # destination of zoom for b
steps_a = (fpoint_a - a_range) / nframes
steps_b = (fpoint_b - b_range) / nframes

# Define a frame update function that gradually zooms in
def mandelbrot_zoom(frame):
    a_vals = np.linspace(a_range[0] + steps_a[0]*frame, a_range[1] + steps_a[1]*frame, n_vals)
    b_vals = np.linspace(b_range[0] + steps_b[0]*frame, b_range[1] + steps_b[1]*frame, n_vals)
    d = np.array([[complex.iterate(z, complex(a, b), max_iter = 100, threshold = 5) \
                   for b in b_vals] for a in a_vals])
    plt.imshow(d.T, cmap = 'Blues_r')
    return

# Save the animation as a gif
anim = FuncAnimation(fig, mandelbrot_zoom, frames = tqdm(range(nframes + 1), position = 0), blit = False)
anim.save('outputs/mandelbrot.gif', writer = 'pillow', fps = 15, dpi = 300)

### Mandelbulb
A 3D extension of the Mandelbrot set, where the iterative function now looks more like $z \rightarrow z^n + c$, where $z$ is approximated as a "triplex" number instead of a complex number. Useful references for reproducing this work are from Daniel White on [Skytopia](https://www.skytopia.com/project/fractal/mandelbulb.html) and Daniel Shiffman's "The Coding Train" [episode](https://www.youtube.com/watch?v=NJCiUVGiNyA).

In [ ]:
# >>> Mandelbulb extension

# Compute the Mandelbulb first
v = np.linspace(-1, 1, 128)
x, y, z = ft.mandelbulb(v, v, v, n = 16, max_iter = 20, threshold = 2)

# Create a 3D figure first
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Formatting
fig.set_facecolor('black')
ax.set_facecolor('black')
ax.w_xaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.w_yaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.w_zaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.grid(False)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(np.min(z), np.max(z))

# Plot points
ax.scatter(x, y, z, color = 'white', s = 0.1)

# Animation settings
nframes = 359 # should be odd to accommodate elevs
azims = np.linspace(0, 360, nframes)
angles = np.linspace(-45, 45, int(np.ceil(nframes/2)))
elevs = np.concatenate((angles, np.flip(angles[:-1])))

# Define a frame update function that rotates
def bulb_rotation(frame):
    ax.view_init(elev = elevs[frame], azim = azims[frame])
    return

# Save the animation as a gif
anim = FuncAnimation(fig, bulb_rotation, frames = tqdm(range(nframes), position = 0), blit = False)
anim.save('outputs/mandelbulb.gif', writer = 'pillow', fps = 30, dpi = 300)